# 3.2 Diseño orientado a objetos para la implementación
### Integrantes: 
1. Albeiro Andino
2. Richard Rocha
3. Xavier Tintin
4. Romina Trujillo


En nuestra introducción a la regresión lineal, repasamos varios componentes, incluidos los datos, el modelo, la función de pérdida y el algoritmo de optimización. De hecho, la regresión lineal es uno de los modelos de aprendizaje automático más simples. Entrenarlo, sin embargo, utiliza muchos de los mismos componentes que requieren otros modelos de este libro. Por lo tanto, antes de sumergirse en los detalles de implementación, vale la pena diseñar algunas de las API utilizadas a lo largo de este libro. Tratando los componentes del aprendizaje profundo como objetos, podemos comenzar definiendo clases para estos objetos y sus interacciones. Este diseño orientado a objetos para la implementación agilizará en gran medida la presentación e incluso es posible que desee usarlo en sus proyectos. Inspirados en bibliotecas de código abierto como PyTorch Lightning67, en un alto nivel deseamos tener tres clases: (i) El módulo contiene modelos, pérdidas y métodos de optimización; (ii) El módulo de datos proporciona cargadores de datos para entrenamiento y validación; (iii) ambas clases se combinan utilizando la clase Trainer, que nos permite entrenar modelos en una variedad de plataformas de hardware. La mayor parte del código de este libro adapta el módulo y el módulo de datos. Solo tocaremos la clase de Trainer cuando hablemos de GPU, CPU, entrenamiento paralelo y algoritmos de optimización.

## 3.2.1 Utilities

Necesitamos algunas utilidades para simplificar la programación orientada a objetos en los cuadernos Jupyter. Uno de los desafíos es que las definiciones de clases tienden a ser bloques de código bastante largos. La legibilidad de los cuadernos requiere fragmentos de código cortos, intercalados con explicaciones, un requisito compatible con el estilo de programación común para las bibliotecas de Python. La primera función de utilidad nos permite registrar funciones como métodos en una clase después de que se haya creado la clase. De hecho, ¡podemos hacerlo incluso después de haber creado instancias de la clase! Nos permite dividir la implementación de una clase en varios bloques de código.

In [ ]:
sub add_to_class {
    my $Class = shift;
    return sub {
        my $obj = shift;
        no strict 'refs';
        *{$Class . '::' . $obj->{name}} = $obj;
    };
}

In [ ]:
# Example:
package MyClass;

my $adder = add_to_class("MyClass");

sub new_method {
    print "This is a new method added at runtime!";
}

$adder->( \&new_method );

MyClass->new_method(); 

# Output: "This is a new method added at runtime!"

Let’s have a quick look at how to use it. We plan to implement a class A with a method do. Instead of having code for both A and do in the same code block, we can first declare the class A and create an instance a.

In [ ]:
package A;

sub new {
    my $class = shift;
    my $self = {
        b => 1
    };
    return bless $self, $class;
}

A continuación, definimos el método como lo haríamos normalmente, pero no en el ámbito de la clase A. En su lugar, decoramos este método con add_to_class con la clase A como argumento. Al hacerlo, el método puede acceder a las variables miembros de A como esperábamos si se hubiera definido como parte de la definición de A. Veamos qué sucede cuando lo invocamos para la instancia a.

In [ ]:
my $adder = add_to_class("A");

$adder->(sub {
    my $self = shift;
    print 'Class attribute "b" is ' . $self->{b};
});

my $a = A->new();
$a->{do}->();

La segunda es una clase de utilidad que guarda todos los argumentos del método _init_ de una clase como atributos de clase. Esto nos permite extender implícitamente las firmas de llamada del constructor sin código adicional.

In [ ]:
sub new {
    my $class = shift;
    my $self = {};
    bless $self, $class;
    return $self;
}

sub save_hyperparameters {
    my ($self, $ignore) = @_;
    $ignore //= [];
    die "NotImplemented";
}

Aplazamos su implementación a la Sección 23.7. Para usarlo, definimos nuestra clase que hereda de HyperParameters y llama a save_hyperparameters en el método _init_.

In [ ]:
package B;
use parent 'HyperParameters';

sub new {
    my ($class, %params) = @_;
    my $self = $class->SUPER::new();
    $self->save_hyperparameters( ignore => ['c'] );
    $self->{a} = $params{a};
    $self->{b} = $params{b};
    print "self.a = $self->{a} self.b = $self->{b}\n";
    print "There is no self.c = ", !exists $self->{c};
    return $self;
}

my $b = B->new(a=>1, b=>2, c=>3);


La última utilidad nos permite trazar el progreso del experimento de forma interactiva mientras se está llevando a cabo. En deferencia al TensorBoard 68 mucho más potente (y complejo), lo nombramos Pro- gressBoard. TheimplementationisdeferredtoSection23.7. Ahora, cojemos la acción 68. La función de dibujo traza un punto (x, y) en la figura, con la etiqueta especificada en la leyenda. El every_n opcional suaviza la línea mostrando solo puntos de 1/n en la figura. Sus valores se calculan a partir de los n puntos de vecino en la figura original.

In [ ]:
package ProgressBoard;
use parent 'HyperParameters';

sub new {
    my ($class, %params) = @_;
    my $self = $class->SUPER::new();
    $self->save_hyperparameters();
    $self->{xlabel} = $params{xlabel} // "";
    $self->{ylabel} = $params{ylabel} // "";
    $self->{xlim} = $params{xlim} // undef;
    $self->{ylim} = $params{ylim} // undef;
    $self->{xscale} = $params{xscale} // "linear";
    $self->{yscale} = $params{yscale} // "linear";
    $self->{ls} = $params{ls} // ['-', '--', '-.', ':'];
    $self->{colors} = $params{colors} // ['C0', 'C1', 'C2', 'C3'];
    $self->{fig} = $params{fig} // undef;
    $self->{axes} = $params{axes} // undef;
    $self->{figsize} = $params{figsize} // [3.5, 2.5];
    $self->{display} = $params{display} // 1;
    return $self;
}

sub draw {
    my ($self, $x, $y, $label, $every_n) = @_;
    $every_n //= 1;
    die "NotImplemented";
}

# my $pb = ProgressBoard->new(xlabel => "X-axis", ylabel => "Y-axis", display => 1);


En el siguiente ejemplo, dibujamos pecado y cos con una suavidad diferente. Si ejecutas este bloque de código, verás que las líneas crecen en la animación.

In [ ]:
use Math::Trig;
use List::Util qw(max);

my $board = ProgressBoard->new( xlabel => 'x');

my @x = map { $_/10 } 0..100;
foreach my $x (@x) {
    $board->draw( $x, sin($x), 'sin', every_n => 2 );
    $board->draw( $x, cos($x), 'cos', every_n => 10 );
}

## 3.2.2 Modelos

La clase de módulo es la clase base de todos los modelos que implementaremos. Como mínimo, necesitamos definir tres métodos. El método _init_ almacena los parámetros aprendibles, el método train- ing_step acepta un lote de datos para devolver el valor de pérdida, el método configure_optimizers devuelve el método de optimización, o una lista de ellos, que se utiliza para actualizar los parámetros aprendibles. Opcionalmente, podemos definir validation_step para informar de las medidas de evaluación.

A veces ponemos el código para calcular la salida en un método de reenvío separado para que sea más reutilizable.

In [ ]:
package Module;
use parent 'nn::Module', 'HyperParameters';

sub new {
    my ($class, %params) = @_;
    my $self = $class->SUPER::new();
    $self->save_hyperparameters();
    $self->{plot_train_per_epoch} = $params{plot_train_per_epoch} // 2;
    $self->{plot_valid_per_epoch} = $params{plot_valid_per_epoch} // 1;
    $self->{board} = ProgressBoard->new();
    return $self;
}

sub loss {
    my ($self, $y_hat, $y) = @_;
    die "NotImplementedError";
}

sub forward {
    my ($self, $X) = @_;
    die "Neural network is defined" unless exists $self->{net};
    return $self->{net}->($X);
}

sub plot {
    my ($self, $key, $value, $train) = @_;
    die "Trainer is not inited" unless exists $self->{trainer};
    $self->{board}->{xlabel} = 'epoch';
    my $x;
    my $n;
    if ($train) {
        $x = $self->{trainer}->{train_batch_idx} / $self->{trainer}->{num_train_batches};
        $n = $self->{trainer}->{num_train_batches} / $self->{plot_train_per_epoch};
    } else {
        $x = $self->{trainer}->{epoch} + 1;
        $n = $self->{trainer}->{num_val_batches} / $self->{plot_valid_per_epoch};
    }
    $self->{board}->draw($x, $value, ('train_' if $train else 'val_') . $key, every_n => int($n));
}

sub training_step {
    my ($self, $batch) = @_;
    my $l = $self->


Puede notar que Module es una subclase de nn.Module, la clase base de redes neuronales en PyTorch. Proporciona funciones convenientes para manejar redes neuronales. Por ejemplo, si definimos un método de avance, como forward(self, X), entonces para una instancia a podemos invocar esta función por a(X). Esto funciona, ya que llama al método de reenvío en el método _call_ integrado. Puede encontrar más detalles y ejemplos sobre nn.Module en la Sección 6.1.

## 3.2.3 Datos

La clase DataModule es la clase base de los datos. Con bastante frecuencia se utiliza el método _init_ para preparar los datos. Esto incluye la descarga y el preprocesamiento si es necesario. El train_dataloader devuelve el cargador de datos para el conjunto de datos de entrenamiento. Un cargador de datos es un generador (Python) que produce un lote de datos cada vez que se utiliza. Este lote se introduce en el método train- ing_step de Module para calcular la pérdida. Hay un val_dataloader opcional para devolver el cargador de conjuntos de datos de validación. Se comporta de la misma manera, excepto que produce lotes de datos para el método validation_step en Module.

In [ ]:
package DataModule;
use parent 'HyperParameters';

sub new {
    my ($class, %params) = @_;
    my $self = $class->SUPER::new();
    $self->save_hyperparameters();
    $self->{root} = $params{root} // '../data';
    $self->{num_workers} = $params{num_workers} // 4;
    return $self;
}

sub get_dataloader {
    my ($self, $train) = @_;
    die "NotImplementedError";
}

sub train_dataloader {
    my ($self) = @_;
    return $self->get_dataloader(train => 1);
}

sub val_dataloader {
    my ($self) = @_;
    return $self->get_dataloader(train => 0);
}


## 3.2.4 Entrenamiento

La clase Trainer entrena los parámetros que se pueden aprender en la clase Module con los datos especificados en DataModule. El método clave es fit, que acepta dos argumentos: modelo, una instancia de Module y data, una instancia de DataModule. Luego itera sobre todo el conjunto de datos max_epochs veces para entrenar el modelo. Como antes, aplazaremos la implementación de esta función a capítulos posteriores.

In [ ]:
package Trainer;
use parent 'HyperParameters';

sub new {
    my ($class, %params) = @_;
    my $self = $class->SUPER::new();
    $self->save_hyperparameters();
    $self->{max_epochs} = $params{max_epochs};
    $self->{num_gpus} = $params{num_gpus} // 0;
    $self->{gradient_clip_val} = $params{gradient_clip_val} // 0;
    die "No GPU support yet" if $self->{num_gpus};
    return $self;
}

sub prepare_data {
    my ($self, $data) = @_;
    $self->{train_dataloader} = $data->train_dataloader();
    $self->{val_dataloader} = $data->val_dataloader();
    $self->{num_train_batches} = scalar @{$self->{train_dataloader}};
    $self->{num_val_batches} = scalar @{$self->{val_dataloader}};
}

sub prepare_model {
    my ($self, $model) = @_;
    $model->{trainer} = $self;
    $model->{board}->{xlim} = [0, $self->{max_epochs}];
    $self->{model} = $model;
}

sub fit {
    my ($self, $model, $data) = @_;
    $self->prepare_data($data);
    $self->prepare_model($model);


## 3.2.5 Resumen

Para resaltar el diseño orientado a objetos para nuestra futura implementación de aprendizaje profundo, las clases anteriores solo muestran cómo sus objetos almacenan datos e interactúan entre sí. Seguiremos enriqueciendo las implementaciones de estas clases, como a través de @add_to_class, en el resto del libro. Además, estas clases totalmente implementadas se guardan en la biblioteca d2l 69, un conjunto de herramientas ligero que facilita el modelado estructurado para el aprendizaje profundo. En particular, facilita la reutilización de muchos componentes entre proyectos sin cambiar mucho en absoluto. Para la postura, podemos reemplazar solo el optimizador, solo el modelo, solo el conjunto de datos, etc.; este grado de modularidad paga dividendos a lo largo del libro en términos de concisión y simplicidad (es por eso que lo agregamos) y puede hacer lo mismo para sus propios proyectos.